# Récupération des jours Tempos

## Objectif
Ce notebook a pour objectif de récupérer les jours Tempo.

Il utilise l'API de Enedis pour récupérer les jours Tempo.

In [24]:
import pandas as pd
import os
import requests
import zipfile
from pathlib import Path

In [8]:
url = "https://eco2mix.rte-france.com/curves/downloadCalendrierTempo?season=14-24"

## Fetch les jours Tempo

Le fichier téléchargé est un fichier CSV mais qui se termine en "xls" et il est zipé.

In [10]:
r = requests.get(url)
root_dir = Path("../")
bronze_dir: Path = root_dir / "data" / "bronze"
filename_zip = bronze_dir / "tempo.zip"
with open(filename_zip, "wb") as f:
    f.write(r.content)

with zipfile.ZipFile(filename_zip, "r") as zip_ref:
    zip_ref.extractall(bronze_dir / "tempo")

## Conversion du fichier


In [22]:
tempo_file = bronze_dir / "tempo" / "eCO2mix_RTE_tempo_2014-2024.xls"
df = pd.read_csv(
    tempo_file,
    sep="\t",
    skipfooter=1,
    engine="python",
    parse_dates=["Date"],
    index_col="Date",
    dtype="category",
)
df.rename(columns={"Type de jour TEMPO": "tempo_type"}, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3584 entries, 2014-09-01 to 2024-06-23
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   tempo_type  3584 non-null   category
dtypes: category(1)
memory usage: 31.6 KB


In [23]:
silver_dir = root_dir / "data" / "silver"
os.makedirs(silver_dir, exist_ok=True)
silver_file = silver_dir / "tempo_2014_2024.csv"
df.to_csv(silver_file)

# Conclusion
Ce notebook utilise l'API de Enedis pour récupérer l'historique des jours Tempo de 2014 à 2024.

Un fichier temporaire est stocké dans le répertoir `/data/bronze` puis la version CSV est stockée dans le répertoire `/data/silver`.
